# Conversion class

In [33]:
import pandas as pd
import numpy as np

class eco_2_5_convert:
    def __init__(self, model_file:str, model_name:str,
                EI_3_8 = False):
        """model_file: the excel model file path
        model_name: a name provided by the user to name the output file and name the model activity
        info_tab: the "info" tab in the eco 2.5 template is a tab where you can add whatever you want
        so it can be used to add calculations and assumptions"""
        
        self.unit_conversion = {'mj': 'megajoule',
                                 'EUR2005': 'EUR2005',
                                 'm': 'meter',
                                 'kg*day': 'kg*day',
                                 'm3': 'cubic meter',
                                 'person kilometer': 'person kilometer',
                                 'meter-year': 'meter-year',
                                 'm2': 'square meter',
                                 'km': 'kilometer',
                                 'l': 'litre',
                                 'kg': 'kilogram',
                                 'm2-year': 'square meter-year',
                                 'kwh': 'kilowatt hour',
                                 'km-year': 'kilometer-year',
                                 'm3-year': 'cubic meter-year',
                                 'unit': 'unit',
                                 'h': 'hour',
                                 'ha': 'hectare',
                                 'bq': 'kilo Becquerel',
                                 'guest night': 'guest night',
                                 'ton kilometer': 'ton kilometer'}
        #Read database ids files
        #ps: make sure all the files are in the same directory you're running jupyter.
        self.activities_raw_ids = pd.read_csv('Activities_with_EI_versions.csv')
        self.products_default = pd.read_csv('[Products]_Id,_[Prod_1673266278497_PRODUCTS.csv')
        self.units =  pd.read_csv('units.csv')
        #Read our LCA custom model
        self.model_activity = pd.read_excel(model_file, sheet_name = 'activity')
        self.model_exchanges = pd.read_excel(model_file, sheet_name = 'exchange')
        self.model_name = model_name
        self.EI_3_8 = EI_3_8

    def convert(self):
        #merge ids with our model to extract all needed ids for the new template
        exchanges_merge = self.model_exchanges.merge(self.activities_raw_ids, left_on = ['product name', 'process name', 'location name'],
                 right_on = ['[products] raw_name', '[activities] name', '[locations] raw_name'])
        self.model_activity['raw_uom'] = self.model_activity['uom'].str.lower().str.strip().map(self.unit_conversion)
        unit_merge = self.model_activity.merge(self.units, left_on = 'raw_uom',
                                              right_on = 'raw_name')
        
        
        location_merge = self.activities_raw_ids.merge(self.model_activity, left_on = '[locations] raw_name',
                                                       right_on = 'location name')
        
        products_processes_merge = exchanges_merge.merge( self.products_default, on = ['[products] raw_name',
                                                                                      '[products] type',
                                                                                       '[categories] id'])
        new_activity  = pd.read_excel('eco 2.5 model upload template.xlsx',
                                      sheet_name = 'activity')
        category_name = self.model_activity['category name'].iloc[0]
        
        category_id = self.activities_raw_ids[self.activities_raw_ids['[categories] raw_name']==category_name]['[categories] id'].iloc[0]
        
        new_activity['Value'] = np.nan
        
        new_activity = new_activity.set_index(['Column','Table'])
        
        new_activity.loc['id', 'source'] = '8b0909fa-a550-11ed-a811-65431102791f'
        
        new_activity.loc['name', 'source'] = 'cei391'

        new_activity.loc['name', 'activity'] = self.model_name
        
        new_activity.loc['unit id', 'activity'] = unit_merge['id'].iloc[0]
        
        new_activity.loc['unit name', 'activity'] = unit_merge['raw_name'].iloc[0]
        
        new_activity.loc['amount', 'activity'] = unit_merge['amount'].iloc[0]
        
        new_activity.loc['location id', 'location'] = location_merge['[locations] id'].iloc[0]
        
        new_activity.loc['location name', 'location'] = location_merge['[locations] raw_name'].iloc[0]
        
        new_activity.loc['name', 'product'] = self.model_activity['product name'].iloc[0]
        
        new_activity.loc['type', 'product'] = self.model_activity['product_type'].iloc[0]
        
        new_activity.loc['name', 'process'] = self.model_activity['process name'].iloc[0]
        
        new_activity.loc['category id', 'product'] = category_id
        
        new_activity.loc['category name', 'product'] = self.model_activity['category name'].iloc[0]
        
        new_activity.loc['description', 'activity'] = self.model_activity['description'].iloc[0]
        
        new_activity.loc['description', 'product'] = self.model_activity['description'].iloc[0]

        #new_activity.loc['name', 'source'] = self.model_activity['database'].iloc[0]

        new_activity.loc['display name en', 'product'] = self.model_activity['product name'].iloc[0]
                
        new_activity.loc['display name en', 'process'] = self.model_activity['process name'].iloc[0]

        


        new_activity = new_activity.reset_index()
        
        
        if self.EI_3_8:
            exchanges_merge['activity id'] = exchanges_merge['EI_3_8_activity_id'].fillna(exchanges_merge['[activities] id'])
            exchanges = exchanges_merge[['activity id', 'EI_3_9_activity_id','amount',
                                         '[products] raw_name', '[processes] raw_name']].drop_duplicates()
            exchanges = exchanges.rename(columns = {'EI_3_8_activity id':'activity id',
                                               '[products] raw_name':'product name',
                                               '[processes] raw_name':'process name'})
        else:
            
            exchanges_merge['activity id'] = exchanges_merge['EI_3_9_activity_id'].fillna(exchanges_merge['[activities] id'])
            exchanges = exchanges_merge[['activity id', 'amount', '[products] raw_name', '[processes] raw_name']].drop_duplicates()
            exchanges = exchanges.rename(columns = {'[products] raw_name':'product name',
                                               '[processes] raw_name':'process name'})
        
        
        
        writer = pd.ExcelWriter('ECO_2_5_{}_model.xlsx'.format(self.model_name), engine='xlsxwriter')
        
        new_activity.to_excel(writer, sheet_name='activity', index = False)
        
        exchanges.to_excel(writer, sheet_name='exchanges', index = False)
        
        writer.close()
        
        return new_activity, exchanges

In [35]:
%%time
#call class with inputs
eco_25 = eco_2_5_convert('transport, freight, aircraft, dedicated freight, very short haul, SAF_.xlsx', 'transport, freight, aircraft, dedicated freight, very short haul, SAF_')

#get outputs from conversion functions
new_activity, exchanges = eco_25.convert()

CPU times: user 169 ms, sys: 51.5 ms, total: 221 ms
Wall time: 255 ms


/Users/burcugulsen/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/burcugulsen/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


# Print

In [17]:
new_activity

,Column,Table,Value
0,id,source,8b0909fa-a550-11ed-a811-65431102791f
1,name,source,cei391
2,name,activity,PV cell electricity
3,unit id,activity,95c4d019-e2da-48d5-a2c4-7f331bcf69ee
4,unit name,activity,kilowatt hour
5,amount,activity,1
6,description,activity,The model represents of 1 kwh electricity prod...
7,default waste activity id,activity,NaN
8,default waste product name,activity,NaN
9,default waste process name,activity,NaN


In [12]:
exchanges

,activity_id,amount,product name,process name
0,c69b0fad-8672-4118-bafa-3d00f8b1187a,7.23590,tap water,market group for tap water
1,ab988f33-5c7b-40d9-94b0-f6173fd7d18c,0.03100,"epoxy resin, liquid","market for epoxy resin, liquid"
2,ab988f33-5c7b-40d9-94b0-f6173fd7d18c,0.00265,"epoxy resin, liquid","market for epoxy resin, liquid"
3,8512273f-db12-4d2e-8951-75a9b641c73a,0.16112,"electricity, medium voltage","market for electricity, medium voltage"


# versions merge

In [1]:
import pandas as pd
activity_versions = pd.read_csv('Activity_id,_Source__1680189888490.csv')
activities_raw_ids = pd.read_csv('[Activities]_Type,_[_1673266308512_ACTIVITIES.csv')

activity_versions_reduced = activity_versions[activity_versions['source_short_name']=='ei38']

activities_two_versions = activity_versions_reduced[['activity_id','version_activity_id']].merge(activities_raw_ids,
                                                                      left_on = 'activity_id', right_on = '[activities] id',
                                                                    how = 'outer')

activities_two_versions = activities_two_versions.rename(columns = {'activity_id':'EI_3_8_activity_id',
                                         'version_activity_id':'EI_3_9_activity_id'})

activities_two_versions.to_csv('Activities_with_EI_versions.csv', index = False)

In [21]:
activities_two_versions[activities_two_versions['[categories] raw_name']=='electricity']['[categories] id'].iloc[0]

array(['a4f4ab9d-35a2-11ec-ba41-b0fc3626dcfb'], dtype=object)